# Notes

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

`!py -3.7 -m pip install PyPDF2`

`!py -3.7 -m pip install tika`

`!py -3.7 -m pip install textract`

`!py -3.7 -m pip install pdfplumber`

# Functions

In [2]:
import pdfplumber

def extract_pdf(pdf_file):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for i in range(len(pdf.pages)):
            first_page = pdf.pages[i]
            #print(first_page.extract_text())
            #f.write(first_page.extract_text())
            text = text + first_page.extract_text()
    return text

In [3]:
def get_ticker(text):
    ticker_start = text.find("EDITED TRANSCRIPT")+18  #EDITED TRANSCRIPT\
    ticker_end = text.find(" ",ticker_start)
    ticker = text[ticker_start:ticker_end]
    #print("Ticker: "+ticker)
    return ticker

In [4]:
def get_presentation(text):    # Need to remove footer, need to remove page count
    presentation = (text[text.find("PRESENTATION")+13:text.find("QUESTIONS AND ANSWERS")]) #PRESENTATION\
    return presentation

In [5]:
months = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']

In [6]:
def get_datetime(text):
    time_start = text.find("EVENT DATE/TIME:")+17
    time_end = text.find("/",time_start)-1
    call_date = text[time_start:time_end] #"EVENT DATE/TIME:"
    year = call_date[-4:]
    day = call_date[call_date.find(" ")+1:call_date.find(",")]
    month = months.index(call_date[:call_date.find(" ")])+1
    #print("Date: "+call_date)
    return [int(year),month,int(day)]

In [7]:
def get_quarter(text_datetime):
    if text_datetime[1] < 4:
        return 1
    if text_datetime[1] < 7:
        return 2
    if text_datetime[1] < 10:
        return 3
    else:
        return 4

In [8]:
import datetime
def get_epoch(text_datetime):
    return int(datetime.datetime(text_datetime[0],text_datetime[1],text_datetime[2]).timestamp())

In [9]:
#Get Price URL Format
#url_eg = "https://query1.finance.yahoo.com/v7/finance/download/<Ticker>?period1=<Start>&period2=<End>&interval=<Number+[d/w/m]>&events=history&includeAdjustedClose=true"
#url_str = "https://query1.finance.yahoo.com/v7/finance/download/{}?period1={}&period2={}&interval=1&events=history&includeAdjustedClose=true" #ticker+1st date+2nd date

def get_csv_url(ticker,text_date):
    epoch1 = get_epoch(text_date) - 604800 # one week before
    epoch2 = get_epoch(text_date) + 604800 # one week after
    ticker = ticker[:ticker.find('.')]
    stock_price = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={epoch1}&period2={epoch2}&interval=1d&events=history&includeAdjustedClose=true" #ticker+1st date+2nd date
    #debug print(stock_price)
    return stock_price

In [10]:
#Merge with csv
import csv
import requests

def get_prices_csv(CSV_URL):

    with requests.Session() as s:
        download = s.get(CSV_URL)

        decoded_content = download.content.decode('utf-8')

        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
        #debug for row in my_list:
            #debug print(row)
        #print(CSV_URL)
        #print(my_list)
        return my_list

In [11]:
def convert_csv_date(date):
    date = date[0].split('-')
    for i in range(3):
        date[i] = int(date[i])
    return date

In [12]:
def get_price_avg_from_csv(csv_list,text_date):
    #Getting days before/since earning calls
    new_columns_to_append_to_row_example = []
    for i in csv_list[1:]:
        average_price = (float(i[1])+float(i[2])+float(i[3])+float(i[4]))/4
        relative_date =  (get_epoch(text_date)-get_epoch(convert_csv_date(i)))/86400
        new_columns_to_append_to_row_example.append(relative_date)
        new_columns_to_append_to_row_example.append(average_price)
    #debug print(new_columns_to_append_to_row_example)
    # in the array, values are days before/since earning call and stock price

    #Price avg of past week, avg of next week
    #replace 0's with nearest neighboor
    is_date = True
    branch = 0
    week_before_price = []
    earning_call_price = []
    week_after_price = []
    for i in new_columns_to_append_to_row_example:
        if is_date:
            is_date = False
            if i > 0:
                branch = 1
            if i == 0:
                branch = 2
            if i < 0:
                branch = 3
        else:
            is_date = True
            if branch == 1:
                week_before_price.append(i)
            if branch == 2:
                earning_call_price.append(i)
            if branch == 3:
                week_after_price.append(i)
    #debug print(week_before_price)
    #debug print(sum(week_before_price)/len(week_before_price))
    #debug print(earning_call_price)
    #debug print(sum(earning_call_price)/len(earning_call_price)) #simplify and account for blanks
    #debug print(week_after_price)
    #debug print(sum(week_after_price)/len(week_after_price))
    return [sum(week_before_price)/len(week_before_price),sum(earning_call_price)/len(earning_call_price),sum(week_after_price)/len(week_after_price)]

In [13]:
def new_row(text,ticker):
    date_array = get_datetime(text)
    quarter = get_quarter(date_array)
    presentation = get_presentation(text)
    csv_url = get_csv_url(ticker,date_array)
    csv_list = get_prices_csv(csv_url)
    prices = get_price_avg_from_csv(csv_list,date_array) #explode?
    str_date = [str(i) for i in date_array] # date convert
    return ['-'.join(str_date),quarter,ticker,presentation,prices[0],prices[1],prices[2]] #row as array

In [14]:
def test_row(text,ticker):
    try:
        date_array = get_datetime(text)
    except:
        date_array = [0, 0, 0]
    try:
        quarter = get_quarter(date_array)
    except:
        quarter = 0
    try:
        presentation = get_presentation(text)
    except:
        presentation = text
    try:
        csv_url = get_csv_url(ticker,date_array)
    except:
        csv_url = "FAILED TO GENERATE"
    try:
        csv_list = get_prices_csv(csv_url)
    except:
        csv_list = []
    try:
        prices = get_price_avg_from_csv(csv_list,date_array) #explode?
    except:
        prices = [0,0,0]
    try:
        str_date = [str(i) for i in date_array] # date convert
    except:
        str_date = ["NA","NA","NA"]
    return ['-'.join(str_date),quarter,ticker,presentation,prices[0],prices[1],prices[2]] #row as array

In [15]:
import pandas as pd
df = pd.DataFrame(columns=['Year-Month-Day','Quarter','Ticker','Text','Price_Prior','Price_During','Price_After'])
df_error = df
df #ADD Category, History, etc...

,Year-Month-Day,Quarter,Ticker,Text,Price_Prior,Price_During,Price_After


## Loop through all transcripts in pdf and store error debugging info

In [16]:
def cut_next_chunck(text):
    brief = text.find("EDITED BRIEF")
    transcript = text.find("EDITED TRANSCRIPT")
    if brief == -1:
        brief = 9999999
    if transcript == -1:
        transcript = 9999999   
    if brief < transcript:
        text = text[brief+2:]
    else:
        text = text[transcript+2:]
    return text

In [17]:
def extract_from_text(text,identifier):
    global df
    global df_error
    text = cut_next_chunck(text)
    while text.find("EDITED TRANSCRIPT") != -1 or text.find("EDITED BRIEF") != -1:
        try: #create row
            a_series = pd.Series(new_row(text,ticker), index = df.columns)
            df = df.append(a_series, ignore_index=True)
        except: #create debugging info
            print("A row failed to be generated")
            a_series =  pd.Series(test_row(text,ticker), index = df_error.columns)
            df_error = df_error.append(a_series, ignore_index=True)
        text = cut_next_chunck(text)
    print(identifier+" ran and finished.") #kill task after that

# Sample function for applying to each file in folder

In [18]:
from multiprocess import Pool, Lock

In [19]:
def init_child(lock_,df_,df_error_):
    global lock
    lock = lock_
    global df
    df = df_
    global df_error
    df_error = df_error_
    

In [20]:
def do_job(filepath):
    import pandas as pd
    import pdfplumber
    
    def extract_pdf(pdf_file):
        text = ""
        with pdfplumber.open(pdf_file) as pdf:
            for i in range(len(pdf.pages)):
                first_page = pdf.pages[i]
                text = text + first_page.extract_text()
        return text
    
    def get_ticker(text):
        ticker_start = text.find("EDITED TRANSCRIPT")+18  #EDITED TRANSCRIPT\
        ticker_end = text.find(" ",ticker_start)
        ticker = text[ticker_start:ticker_end]
        if "." not in ticker[-3:]: #PROVISORELY FIXING BUG WITH READING TICKER
            ticker = ticker+".N"
        return ticker
    
    def get_presentation(text):    # Need to remove footer, need to remove page count
        presentation = (text[text.find("PRESENTATION")+13:text.find("QUESTIONS AND ANSWERS")]) #PRESENTATION\
        return presentation
    
    months = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']
    
    def get_datetime(text):
        time_start = text.find("EVENT DATE/TIME:")+17
        time_end = text.find("/",time_start)-1
        call_date = text[time_start:time_end] #"EVENT DATE/TIME:"
        year = call_date[-4:]
        day = call_date[call_date.find(" ")+1:call_date.find(",")]
        month = months.index(call_date[:call_date.find(" ")])+1
        #print("Date: "+call_date)
        return [int(year),month,int(day)]
    
    def get_quarter(text_datetime):
        if text_datetime[1] < 4:
            return 1
        if text_datetime[1] < 7:
            return 2
        if text_datetime[1] < 10:
            return 3
        else:
            return 4
    
    import datetime
    def get_epoch(text_datetime):
        return int(datetime.datetime(text_datetime[0],text_datetime[1],text_datetime[2]).timestamp())
    
    def get_csv_url(ticker,text_date):
        epoch1 = get_epoch(text_date) - 604800 # one week before
        epoch2 = get_epoch(text_date) + 604800 # one week after
        ticker = ticker[:ticker.find('.')]
        stock_price = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={epoch1}&period2={epoch2}&interval=1d&events=history&includeAdjustedClose=true" #ticker+1st date+2nd date
        return stock_price
    
    import csv
    import requests
    def get_prices_csv(CSV_URL):
        with requests.Session() as s:
            download = s.get(CSV_URL)
            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            return my_list
    
    def convert_csv_date(date):
        date = date[0].split('-')
        for i in range(3):
            date[i] = int(date[i])
        return date
    
    def get_price_avg_from_csv(csv_list,text_date):
        new_columns_to_append_to_row_example = []
        for i in csv_list[1:]:
            average_price = (float(i[1])+float(i[2])+float(i[3])+float(i[4]))/4
            relative_date =  (get_epoch(text_date)-get_epoch(convert_csv_date(i)))/86400
            new_columns_to_append_to_row_example.append(relative_date)
            new_columns_to_append_to_row_example.append(average_price)

        is_date = True
        branch = 0
        week_before_price = []
        earning_call_price = []
        week_after_price = []
        for i in new_columns_to_append_to_row_example:
            if is_date:
                is_date = False
                if i > 0:
                    branch = 1
                if i == 0:
                    branch = 2
                if i < 0:
                    branch = 3
            else:
                is_date = True
                if branch == 1:
                    week_before_price.append(i)
                if branch == 2:
                    earning_call_price.append(i)
                if branch == 3:
                    week_after_price.append(i)
        return [sum(week_before_price)/len(week_before_price),sum(earning_call_price)/len(earning_call_price),sum(week_after_price)/len(week_after_price)]
    
    def new_row(text,ticker):
        date_array = get_datetime(text)
        quarter = get_quarter(date_array)
        presentation = get_presentation(text)
        csv_url = get_csv_url(ticker,date_array)
        csv_list = get_prices_csv(csv_url)
        prices = get_price_avg_from_csv(csv_list,date_array) #explode?
        str_date = [str(i) for i in date_array] # date convert
        return ['-'.join(str_date),quarter,ticker,presentation,prices[0],prices[1],prices[2]] #row as array
    
    def test_row(text,ticker):
        try:
            date_array = get_datetime(text)
        except:
            date_array = [0, 0, 0]
        try:
            quarter = get_quarter(date_array)
        except:
            quarter = 0
        try:
            presentation = get_presentation(text)
        except:
            presentation = text
        try:
            csv_url = get_csv_url(ticker,date_array)
        except:
            csv_url = "FAILED TO GENERATE"
        try:
            csv_list = get_prices_csv(csv_url)
        except:
            csv_list = []
        try:
            prices = get_price_avg_from_csv(csv_list,date_array) #explode?
        except:
            prices = [0,0,0]
        try:
            str_date = [str(i) for i in date_array] # date convert
        except:
            str_date = ["NA","NA","NA"]
        return ['-'.join(str_date),quarter,ticker,presentation,prices[0],prices[1],prices[2]] #row as array
    
    def cut_next_chunck(text):
        brief = text.find("EDITED BRIEF")
        transcript = text.find("EDITED TRANSCRIPT")
        if brief == -1:
            brief = 9999999
        if transcript == -1:
            transcript = 9999999   
        if brief < transcript:
            text = text[brief+2:]
        else:
            text = text[transcript+2:]
        return text
    
    def extract_from_text(text,identifier):
        global df
        global df_error
        text = cut_next_chunck(text)
        while text.find("EDITED TRANSCRIPT") != -1 or text.find("EDITED BRIEF") != -1:
            try: #create row
                a_series = pd.Series(new_row(text,ticker), index = df.columns)
                df = df.append(a_series, ignore_index=True)
            except: #create debugging info
                print("A row failed to be generated")
                a_series =  pd.Series(test_row(text,ticker), index = df_error.columns)
                df_error = df_error.append(a_series, ignore_index=True)
            text = cut_next_chunck(text)
        print(identifier+" ran and finished.") #kill task after that
    
    text = extract_pdf(filepath)
    ticker = get_ticker(text)
    extract_from_text(text,"alpha")
    return [df, df_error]

def create_pool(filepaths):
    filepaths = filepaths #[:6]
    global dfs
    global to_concat
    global to_concat_error
    lock = Lock()
    poolsize = 6
    with Pool(poolsize, initializer=init_child, initargs=(lock,df,df_error,)) as pool:
        print(filepaths)
        dfs = pool.imap_unordered(do_job, filepaths)
        to_concat = []
        to_concat_error = []
        
        for dftuple in dfs:
            to_concat.append(dftuple[0])
            to_concat_error.append(dftuple[1])
        print("DONE on create_poll")

In [21]:
import glob
# Print pdf files in folder
dfs = []
filepaths = glob.glob(r'D:/Downloads/Conference Call data 2/*.pdf')
create_pool(filepaths)
print("It is DONE")
#do_job(filepaths[0])

['D:/Downloads/Conference Call data 2\\AAPL.pdf', 'D:/Downloads/Conference Call data 2\\ABBV_1.pdf', 'D:/Downloads/Conference Call data 2\\ABBV_2.pdf', 'D:/Downloads/Conference Call data 2\\ABT_1.pdf', 'D:/Downloads/Conference Call data 2\\ACN.pdf', 'D:/Downloads/Conference Call data 2\\ADBE_1.pdf', 'D:/Downloads/Conference Call data 2\\AMGN_1.pdf', 'D:/Downloads/Conference Call data 2\\AMGN_2.pdf', 'D:/Downloads/Conference Call data 2\\AMGN_3.pdf', 'D:/Downloads/Conference Call data 2\\AVGO.pdf', 'D:/Downloads/Conference Call data 2\\BAC_1.pdf', 'D:/Downloads/Conference Call data 2\\BAC_2.pdf', 'D:/Downloads/Conference Call data 2\\BA_1.pdf', 'D:/Downloads/Conference Call data 2\\BA_2.pdf', 'D:/Downloads/Conference Call data 2\\BMY_1.pdf', 'D:/Downloads/Conference Call data 2\\BMY_2.pdf', 'D:/Downloads/Conference Call data 2\\CAT_1.pdf', 'D:/Downloads/Conference Call data 2\\CMCSA_1.pdf', 'D:/Downloads/Conference Call data 2\\CMCSA_2.pdf', 'D:/Downloads/Conference Call data 2\\COST_1.

# Prototype of code for downloading price for stock and converting dates

In [22]:
df = pd.concat(to_concat)
df

,Year-Month-Day,Quarter,Ticker,Text,Price_Prior,Price_During,Price_After
0,2018-5-9,2,ABBV.N,"Gregory B. Gilbert - Deutsche Bank AG, Researc...",100.064999,100.390001,104.073125
1,2018-4-26,2,ABBV.N,"Operator\nGood morning, and thank you for stan...",92.584500,95.892502,99.713126
2,2018-4-26,2,ABBV.N,SUMMARY\nI. 1Q18 REVIEW (R.G.)\nA. Highlights:...,92.584500,95.892502,99.713126
3,2018-3-15,1,ABBV.N,Geoffrey Christopher Meacham - Barclays Bank P...,118.544500,117.077499,113.309375
4,2018-1-26,1,ABBV.N,"Operator\nGood morning, and thank you for stan...",105.629500,119.267500,117.132500
...,...,...,...,...,...,...,...
743,2017-4-28,2,XOM.N,"Operator\nGood day, everyone, and welcome to t...",81.263501,82.039999,82.080000
744,2017-4-28,2,XOM.N,SUMMARY\nI. 1Q17 REVIEW (J.W.)\nA. Headlines:\...,81.263501,82.039999,82.080000
745,2017-3-1,1,XOM.N,"Jeff Woodbury - ExxonMobil Corporation - VP, I...",81.396000,82.437498,82.895001
746,2017-1-31,1,XOM.N,"Operator\nGood day, everyone, and welcome to t...",85.296500,84.282499,83.355001


In [23]:
df_error = pd.concat(to_concat_error)
df_error

,Year-Month-Day,Quarter,Ticker,Text,Price_Prior,Price_During,Price_After
0,2021-1-30,1,AMGN.OQ,"Operator\nMy name is Cindy, and I'll be your c...",0,0,0
1,2020-9-20,3,AMGN.OQ,"Operator\nGood afternoon. My name is Annie, an...",0,0,0
2,2019-9-28,3,AMGN.OQ,"Operator\nMy name is Mary, and I will be your ...",0,0,0
3,2019-9-8,3,AMGN.OQ,"Operator\nMy name is Mary, and I will be your ...",0,0,0
0,2017-12-9,4,AMGN.N,"Arvind Sood - Amgen Inc. - VP of IR\nOkay, I t...",0,0,0
...,...,...,...,...,...,...,...
5,2018-12-5,4,C.N,Richard Nigel Ramsden - Goldman Sachs Group In...,0,0,0
6,2017-6-3,2,LLY.N,"Operator\nLadies and gentlemen, thank you for ...",0,0,0
7,2017-6-3,2,LLY.N,SUMMARY\nI. INTRODUCTION (S.M.)\nA. Overview:\...,0,0,0
8,2017-6-10,2,MDT.N,Ryan Weispfenning - Medtronic plc - VP of IR\n...,0,0,0


In [24]:
df.to_excel("CallData1V9.xlsx")

In [25]:
df_error.to_excel("CallData1V9Errors.xlsx")  